# Composing Contracts II

Toying with ideas from [How to Write a Financial Contract](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.14.7885) by S. L. Peyton Jones and J-M. Eber. There is an old version of the paper [Composing Contracts:An Adventure in Financial Engineering](https://www.cs.tufts.edu/~nr/cs257/archive/simon-peyton-jones/contracts.pdf) by Jones, Eber and Seward. Some code for this version can be found in [Composing Contracts](./Composing%20Contracts.ipynb).

In [1]:
from typing import TypeVar, Generic
from dataclasses import dataclass, asdict, astuple
from abc import ABC, abstractmethod
from datetime import datetime

In [2]:
class ObservableFloat(ABC):
    pass

class ObservableBool(ABC):
    pass

In [3]:
@dataclass
class Konst(ObservableFloat):
    constant: float

@dataclass
class Stock(ObservableFloat):
    ticker: str

In [4]:
T = TypeVar('T')

class ObservableFloatVisitor(ABC, Generic[T]):
    
    @abstractmethod
    def konst(self, constant: float) -> T: pass

    def stock(self, ticker: str) -> T: pass
    
    def __call__(self, observable: ObservableFloat) -> T:
        if isinstance(observable, Konst):
            return self.konst(observable.constant)
        elif isinstance(observable, Stock):
            return self.stock(observable.ticker)
        else:
            raise TypeError(f'Unknown observable float type "{type(contract).__name__}"')

In [5]:
class Contract(ABC):
    pass

In [6]:
@dataclass
class Zero(Contract):
    pass

@dataclass
class One(Contract):
    currency: str

@dataclass
class Give(Contract):
    contract: Contract

@dataclass
class And(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Or(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Cond(Contract):
    observable: ObservableBool
    contract1: Contract
    contract2: Contract    

@dataclass
class Scale(Contract):
    observable: ObservableFloat
    contract: Contract

@dataclass
class When(Contract):
    observable: ObservableBool
    contract: Contract

@dataclass
class Anytime(Contract):
    observable: ObservableBool
    contract: Contract

@dataclass
class Until(Contract):
    observable: ObservableBool
    contract: Contract

In [7]:
T = TypeVar('T')

class ContractVisitor(ABC, Generic[T]):

    @abstractmethod
    def zero(self) -> T: pass

    @abstractmethod
    def one(self, currency: str) -> T: pass

    @abstractmethod
    def give(self, contract: Contract) -> T: pass

    @abstractmethod
    def and_(self, contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def or_(self, contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def cond(self, observable: ObservableBool, contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def scale(self, observable: ObservableFloat, contract: Contract) -> T: pass

    @abstractmethod
    def when(self, observable: ObservableBool, contract: Contract) -> T: pass

    @abstractmethod
    def anytime(self, observable: ObservableBool, contract: Contract) -> T: pass

    @abstractmethod
    def until(self, observable: ObservableBool, contract: Contract) -> T: pass

    def __call__(self, contract: Contract) -> T:
        if isinstance(contract, Zero):
            return self.zero()
        elif isinstance(contract, One):
            return self.one(**contract.__dict__)
        elif isinstance(contract, Give):
            return self.give(**contract.__dict__)
        elif isinstance(contract, And):
            return self.and_(**contract.__dict__)
        elif isinstance(contract, Or):
            return self.or_(**contract.__dict__)
        elif isinstance(contract, Cond):
            return self.cond(**contract.__dict__)
        elif isinstance(contract, Scale):
            return self.scale(**contract.__dict__)
        elif isinstance(contract, When):
            return self.when(**contract.__dict__)
        elif isinstance(contract, Anytime):
            return self.anytime(**contract.__dict__)
        elif isinstance(contract, Until):
            return self.until(**contract.__dict__)
        else:
            raise TypeError(f'Unknown contract type "{type(contract).__name__}"')